In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import uproot
import awkward as ak
import pandas as pd


# Loading roofile things into data frames (no osc samples)

In [ ]:
loc = "BNBNuMI_TOsc_input_numifluxpatched/hist_rootfiles/"

beams = [
    "BNB", 
    "NuMI",
]

runs = [
    "run1",
    "run2",
    "run3",
]

horns = [
    "FHC",
    "RHC",
]

sels = [
    ("nueCC", "_FC"),
    ("nueCC", "_PC"),
    ("numuCC", "_FC"),
    ("numuCC", "_PC"),
    ("CCpi0", "_FC"),
    ("CCpi0", "_PC"),
    ("NCpi0", ""),
]

filetypes = [
    ("intrinsic", "intnue"), 
    ("nu_overlay", "overlaynumu"),
]

dfs = []

for beam in beams:
    for filetype_pair in filetypes:
        for run in runs:
            for horn in horns:

                if beam == "NuMI":
                    runhorn = run + "_" + horn
                else:
                    runhorn = run

                if runhorn == "run3_FHC":
                    continue

                filename = f"roofile_obj_{beam}_{runhorn}_{filetype_pair[0]}.root"
                #print(loc+filename)
                f = uproot.open(loc+filename)

                for sel in sels:

                    try:
                        key = f"tree_{sel[0]}_from_{filetype_pair[1]}{sel[1]}"
                        #print(f"    key: {key}, {f[key].keys()}")
                        #print(f[key])

                        curr_df = pd.DataFrame()
                        for subkey in f[key].keys():
                            curr_df[subkey] = f[key][subkey].array(library="pd")

                        curr_df["beam"] = beam
                        curr_df["filetype"] = filetype_pair[0]
                        curr_df["run"] = run
                        curr_df["horn"] = horn
                        curr_df["sel"] = sel[0] + sel[1]

                        dfs.append(curr_df)
                    except KeyError:
                        #print(f"    KeyError: {key}")
                        pass

all_df = pd.concat(dfs, ignore_index=True, axis=0)

all_df

